## Autoencoders linear

O objetivo desse exemplo é prever se o cliente vai pagar ou não o empréstimo.

Estamos utilizando o banco de crédito obtido pelo Kaggle: https://www.kaggle.com/c/home-credit-default-risk.

Iremos realizar uma comparação ao final de prever isso com estimator vs autoencoder.

In [28]:
# Realizando as importações
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.contrib.layers import fully_connected

### Inicialmente vamos realizar a leitura e tratamento dos dados

In [2]:
base = pd.read_csv('credit_data.csv')
base.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [3]:
# Observando dimensões da base
base.shape 

(2000, 5)

In [4]:
# Excluindo as linhas que tem NA
base = base.dropna()
base.shape

(1997, 5)

In [5]:
# Retirando a coluna'i#clientid'
base = base.drop('i#clientid', axis = 1)
base.head()

,income,age,loan,c#default
0,66155.925095,59.017015,8106.532131,0
1,34415.153966,48.117153,6564.745018,0
2,57317.170063,63.108049,8020.953296,0
3,42709.534201,45.751972,6103.642260,0
4,66952.688845,18.584336,8770.099235,1


In [6]:
# Deixando os dados padronizados
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
base.head()

,income,age,loan,c#default
0,1.453898,1.336861,1.201907,0
1,-0.762398,0.536639,0.695744,0
2,0.836733,1.637207,1.173812,0
3,-0.183244,0.362998,0.544366,0
4,1.509532,-1.631534,1.419754,1


In [7]:
# Separando os atributos (X) dos labels/rótulos (Y)
X = base.drop('c#default', axis = 1)
Y = base['c#default']

In [8]:
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [9]:
Y.head()

0    0
1    0
2    0
3    0
4    1
Name: c#default, dtype: int64

In [10]:
# Criando as colunas
colunas = [tf.feature_column.numeric_column(key = column) for column in X.columns]

print("Colunas de X: ", X.columns)
print("\nColunas: \n", colunas)

print("\n\n Colunas em partes:")
print("\n ", colunas[0])
print("\n ", colunas[1])
print("\n ", colunas[2])

Colunas de X:  Index(['income', 'age', 'loan'], dtype='object')

Colunas: 
 [NumericColumn(key='income', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='loan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


 Colunas em partes:

  NumericColumn(key='income', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

  NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

  NumericColumn(key='loan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [11]:
# Separando conjunto de dados em treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

# Analisando as dimensões
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("Y_train: ", Y_train.shape)
print("Y_test: ", Y_test.shape)

X_train:  (1397, 3)
X_test:  (600, 3)
Y_train:  (1397,)
Y_test:  (600,)


### Utilizando o estimator, teremos:


In [12]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_train,
                                                        y = Y_train,
                                                        batch_size = 8, 
                                                        num_epochs = None,
                                                        shuffle = True)

classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4])
classificador.train(input_fn = funcao_treinamento, steps = 1000)

funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_test, y = Y_test,
                                              batch_size = 8, num_epochs = 1000,
                                              shuffle = False)

metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)
print("\n")
print(metricas_teste)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Natielle\\AppData\\Local\\Temp\\tmpsju0e3m1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000013297EDC7F0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:


## Aplicando o autoencoder

In [13]:
# Criando as quantidades de neurônios
neuronios_entrada = 3
neuronios_oculta = 2
neuronios_saida = neuronios_entrada
# 3 -> 2: Estamos reduzindo a dimensionalidade
# 2 -> 3: Estamos aumentando a dimensionalidade

In [14]:
# Criando o placeholder
xph = tf.placeholder(tf.float32, shape = [None, neuronios_entrada])

In [18]:
# Criando os autoenconders
# fully_connected -> significa que é uma rede neural densa, onde todos neurônios conectam com todos da próxima camada
camada_oculta = fully_connected(inputs = xph, # camada de entrada, pois a camada anterior a camada oculta é a de entrada
                                num_outputs = neuronios_oculta, 
                                activation_fn = None) # não estamos realizando a aplicação de função de ativação
camada_saida = fully_connected(inputs = camada_oculta, # pois a camada anterior a camada de saída é a oculta
                               num_outputs = neuronios_saida)  

In [19]:
# Calculando o erro
erro = tf.losses.mean_squared_error(labels = xph, # vamos comparar com a camada de entrada 
                                    predictions = camada_saida)
otimizador = tf.train.AdamOptimizer(0.01)
treinamento = otimizador.minimize(erro) # Minimizando o erro

In [23]:
# Executando tudo de fato
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # realizando o treinamento
    for epoca in range(1000):
        custo, _ = sess.run([erro, treinamento], feed_dict = {xph: X})
        if epoca % 100 == 0:
            print('erro: ' + str(custo))
            
    print('\nPodemos observar que o erro vai diminuindo com o treinamento.\n')
            
    # 
    X2d_encode = sess.run(camada_oculta, feed_dict = {xph: X})
    X3d_decode = sess.run(camada_saida, feed_dict = {xph: X})
    
    print("\nX2d_encode: \n", X2d_encode)
    print("\n\nX3d_decode: \n", X3d_decode)

erro: 1.5069944
erro: 0.63135266
erro: 0.58495986
erro: 0.57598567
erro: 0.56560844
erro: 0.5622058
erro: 0.56118125
erro: 0.5606759
erro: 0.560444
erro: 0.5603169

Podemos observar que o erro vai diminuindo com o treinamento.


X2d_encode: 
 [[ 0.45600998 -1.9151399 ]
 [-0.61302245 -0.7591479 ]
 [-0.32221293 -2.4142435 ]
 ...
 [ 1.2235609   1.5854264 ]
 [-2.4051116  -2.590667  ]
 [ 0.69061357 -1.5787902 ]]


X3d_decode: 
 [[1.1692138  1.3354464  1.5597486 ]
 [0.16800143 0.54655224 0.        ]
 [0.92065716 1.6398203  1.0573995 ]
 ...
 [0.33660334 0.         0.        ]
 [0.         1.6945755  0.        ]
 [1.1775469  1.1218553  1.5665538 ]]


Podemos observar que X2d_encode possui 2 dimensões, enquanto que o X3d_decode possui 3 dimensões.

Ao observarmos X2d_encode, vemos os dados codificados.

Ao observarmos X3d_decode, vemos uma aproximação dos dados de entradas.

In [24]:
print("Dimensão de X2d_encode:", X2d_encode.shape)
print("Dimensão de X3d_decode:", X3d_decode.shape)

Dimensão de X2d_encode: (1997, 2)
Dimensão de X3d_decode: (1997, 3)


In [26]:
# Agora vamos despadronizar os dados reais e os previsto
X2 = scaler_x.inverse_transform(X)
print("Despadronizando os dados reais...\n", X2)

X3d_decode2 = scaler_x.inverse_transform(X3d_decode)
print("\n\nDespadronizando os dados previstos...\n", X3d_decode2)

Despadronizando os dados reais...
 [[6.61559251e+04 5.90170151e+01 8.10653213e+03]
 [3.44151540e+04 4.81171531e+01 6.56474502e+03]
 [5.73171701e+04 6.31080495e+01 8.02095330e+03]
 ...
 [4.43114493e+04 2.80171669e+01 5.52278669e+03]
 [4.37560566e+04 6.39717958e+01 1.62272260e+03]
 [6.94365796e+04 5.61526170e+01 7.37883360e+03]]


Despadronizando os dados previstos...
 [[6.2078812e+04 5.8997749e+01 9.1965283e+03]
 [4.7739902e+04 4.8252178e+01 4.4454878e+03]
 [5.8519098e+04 6.3143646e+01 7.6663579e+03]
 ...
 [5.0154543e+04 4.0807560e+01 4.4454878e+03]
 [4.5333863e+04 6.3889469e+01 4.4454878e+03]
 [6.2198152e+04 5.6088413e+01 9.2172568e+03]]


In [29]:
# Calculando os erros APLICANDO NOS DADOS SEM REDUÇÃO DA DIMENSIONALIDADE
mae_income = mean_absolute_error(X2[:,0], X3d_decode2[:,0])
print("\nErro absoluto sobre o atributo income: ", mae_income)

mae_age = mean_absolute_error(X2[:,1], X3d_decode2[:,1])
print("\nErro absoluto sobre o atributo age: ", mae_age)

mae_loan = mean_absolute_error(X2[:,2], X3d_decode2[:,2])
print("\nErro absoluto sobre o atributo loan: ", mae_loan)


Erro absoluto sobre o atributo income:  9416.715524792195

Erro absoluto sobre o atributo age:  5.870525778826694

Erro absoluto sobre o atributo loan:  1799.1941119359963


In [31]:
# AGORA VAMOS APLICAR O ALGORITMO COM OS DADOS COM DIMENSIONALIDADE REDUZIDA
X_encode = pd.DataFrame({'atributo1': X2d_encode[:,0], 'atributo2': X2d_encode[:,1], 'classe': Y})
X_encode.head()

,atributo1,atributo2,classe
0,0.456010,-1.915140,0
1,-0.613022,-0.759148,0
2,-0.322213,-2.414243,0
3,-0.109995,-0.464085,0
4,3.768131,2.760334,1


In [32]:
# TREINANDO E CLASSIFICANDO COM OS DADOS DE 2 DIMENSÕES (DIMENSÕES REDUZIDAS)
colunas = [tf.feature_column.numeric_column(key = column) for column in X_encode.columns]

# Separando em treino e teste
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X_encode, Y, 
                                                                  test_size = 0.3, random_state = 0)

# Treinando
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento,
                                                        y = Y_treinamento,
                                                        batch_size = 8, 
                                                        num_epochs = None,
                                                        shuffle = True)

# Criando o classificador
classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4])
classificador.train(input_fn = funcao_treinamento, steps = 1000)

# Testando
funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_teste, y = Y_teste,
                                              batch_size = 8, num_epochs = 1000,
                                              shuffle = False)

# Calculando os erros aplicando nos dados COM REDUÇÃO DA DIMENSIONALIDADE
metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)
metricas_teste






INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Natielle\\AppData\\Local\\Temp\\tmpxu5mpped', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001329C971DA0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Che

{'accuracy': 1.0,
 'accuracy_baseline': 0.8635,
 'auc': 1.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.001915068,
 'label/mean': 0.1365,
 'loss': 0.015320544,
 'precision': 1.0,
 'prediction/mean': 0.13701093,
 'recall': 1.0,
 'global_step': 1000}

Podemos observar que quando aplicamos nos dados com dimensão reduzida, o desempenho (neste caso e com o parâmetros que utilizamos) é melhor do que quando aplicamos nos dados com as dimensões originais.